In [1]:
from tensorflow.keras.layers import Dense, LSTM, Conv2D, TimeDistributed, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import numpy as np

In [2]:
config = tf.compat.v1.ConfigProto(gpu_options= tf.compat.v1.GPUOptions(allow_growth=True))
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

Num GPUs: 1


In [3]:
frames = 300
x = 240
y = 320
classes = ["Reading", "Drink", "Eat", "Call Cellphone", "Write", "Use Laptop", "Use Vaccum Cleaner", "Cheer Up", "Sit Still", "Toss Paper", "Play Game", "Lay Down", "Walk", "Stand Up", "Sit Down"]
outputs = len(classes)


In [5]:
model = Sequential()
model.add(TimeDistributed(Conv2D(input_shape=(frames, x, y, 1), filters=64,kernel_size=(3,3),padding="same", activation="relu")))
#model.add(TimeDistributed(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")))
model.add(TimeDistributed(GlobalAveragePooling2D()))
model.add(LSTM(frames, input_shape=(frames,1), activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(10)))
#model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(outputs, activation='softmax'))

In [6]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.build((320,frames,x,y,1))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_3 (TimeDis  (320, 300, 240, 320, 64)  640      
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (320, 300, 64)           0         
 tributed)                                                       
                                                                 
 lstm_1 (LSTM)               (320, 300, 300)           438000    
                                                                 
 time_distributed_5 (TimeDis  (320, 300, 10)           3010      
 tributed)                                                       
                                                                 
 dense_5 (Dense)             (320, 300, 512)           5632      
                                                      

In [ ]:
import cv2 as cv
import os

# for f in os.listdir("MSRDailyAct3D_pack2/Reading"):
#     if f.endswith(".avi"):
#         continue
#     os.remove(os.path.join("MSRDailyAct3D_pack2/Reading", f))

i=0
c=0
X_train = []
Y_train = []
maxx=0
for path in os.listdir("C:/Users/admin/Dharmik Mohil/MSR_U"):
    for path1 in os.listdir("C:/Users/admin/Dharmik Mohil/MSR_U/"+path):
        capture = cv.VideoCapture("C:/Users/admin/Dharmik Mohil/MSR_U/"+path+"/"+path1)
                    
        i = i+1   

        temp_frames = []
        while(True):

            ret, frame = capture.read()
            #print(ret)
            if ret == True:
                
                scale_percent = 50 # percent of original size
                width = int(frame.shape[1] * scale_percent / 100)
                height = int(frame.shape[0] * scale_percent / 100)
                dim = (width, height)
  
                resized = cv.resize(frame, dim, interpolation = cv.INTER_AREA)
                grayImage = cv.cvtColor(resized, cv.COLOR_BGR2GRAY)
#                 cv.imshow('Frame '+ str(i), grayImage)

                if(len(temp_frames)<frames):
                    temp_frames.append(grayImage)


            # Break the loop
            else: 
                break

        for t in range(frames-len(temp_frames)):
            temp_frames.append(np.zeros((x,y)))
        X = np.array(temp_frames)
        X=X.reshape(X.shape[0],X.shape[1],X.shape[2],1)
        X=X.astype("float16")
        ind = 0
        label = np.zeros((X.shape[0], len(classes)))
        label[:][ind] = 1
        Y_train.append(label)
        #print(label.shape)
        X_train.append(X)


capture.release()
cv.destroyAllWindows()

In [19]:
maxx

800

In [11]:
X_train_final = np.array(X_train)
Y_train_final = np.array(Y_train)

# model.fit(X_train_final, Y_train_final)

In [10]:
X_train_final.shape

(320, 300, 240, 320, 1)

In [12]:
model.fit(X_train_final, Y_train_final, batch_size=1, epochs=5)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
model.save('HAR_weights.h5')